
# FrankPEPstein: Interactive Peptide Fragment Design

This notebook implements the FrankPEPstein pipeline for designing peptide fragments binding to a specific protein pocket.

**Steps:**
1.  Setup & Dependencies
2.  Input Data (Receptor)
3.  Pocket Selection
4.  Fragment Generation & Ranking


In [ ]:

#@title Factory Reset (Uncomment and run to wipe runtime)
# from google.colab import runtime
# runtime.unassign()


## 0. Setup & Dependencies

In [ ]:

#@title 0.1 Install CondaColab (Running this will restart the session)
import sys
import os

# Helper to suppress output
class SuppressStdout:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

try:
    import condacolab
    with SuppressStdout():
        condacolab.check()
except ImportError:
    print("Installing CondaColab...")
    !pip install -q condacolab
    import condacolab
    with SuppressStdout():
        condacolab.install()
print("CondaColab installed.")


In [ ]:

#@title 0.2 Install Dependencies & Setup Tools
#@markdown This cell clones the repository, installs bio-dependencies, and sets up external tools.

import os
import sys
import subprocess

# --- 1. Clone Repository ---
if not os.path.exists("FrankPEPstein"):
    print("Cloning repository...")
    !git clone https://github.com/Joacaldog/FrankPEPstein.git
else:
    print("Repository already exists.")

# --- 2. Install Core Bio-Dependencies ---
print("Installing bio-dependencies (this may take a few minutes)...")
!mamba install -q -c conda-forge -c salilab openbabel biopython fpocket joblib tqdm py3dmol vina python=3.10 salilab::modeller

# --- 3. Setup External Tools & Config ---
repo_path = os.path.abspath("FrankPEPstein")
if repo_path not in sys.path:
    sys.path.append(repo_path)
from scripts import notebook_utils

# DRIVE CONFIGURATION: Enter your File IDs here
drive_ids = {
    "adfr_id": "1gmRj8mva84-JB7UXUcQfB3Ziw_nwwdox",       # ADFRsuite_x86_64Linux_1.0.tar.gz
    "db_id": "1a4GoZ1ZT-DNYMyvVtKJukNdF6TAaLJU5",    # minipockets_..._curated-db.tar.gz
    "dict_id": "1nrwSUof0lox9fp8Ow5EICIN9u0lglu7U"      # reduce_wwPDB_het_dict.tar.gz
}

print("Setting up external tools...")
notebook_utils.setup_external_tools(drive_ids)

print("Configuring Modeller...")
notebook_utils.configure_modeller()

print("Setup Complete!")


## 1. Input Data: Upload Receptor

In [ ]:

from google.colab import files
import os

print("Upload your Receptor PDB file (e.g., receptor.pdb)")
uploaded = files.upload()
receptor_filename = list(uploaded.keys())[0]
print(f"Receptor uploaded: {receptor_filename}")
